## Sequence-Length Dataset

In [1]:
import numpy as np
from transformers import AutoTokenizer
from datasets import Dataset, load_from_disk

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-13b-hf")
tokenizer.pad_token = tokenizer.eos_token

token_id = 319 # depends on tokenizer
print(tokenizer.decode(token_id))

A


In [2]:
N = 512   # for train / test
L = 16384 # for instruction + response

ds_path = f"../dataset/DEBUG_N{N}_L{L}"
print(ds_path)

instructions = np.ones((N * 2, L // 2), dtype=np.int32) * token_id
outputs = np.ones((N * 2, L // 2), dtype=np.int32) * token_id
texts = np.ones((N * 2, L), dtype=np.int32) * token_id

instruction_texts = tokenizer.batch_decode(instructions)
output_texts = tokenizer.batch_decode(outputs)
texts = tokenizer.batch_decode(texts)

data = dict(instruction=instruction_texts, output=output_texts, text=texts)
ds = Dataset.from_dict(data).train_test_split(test_size=N)
ds.save_to_disk(ds_path)

# check from disk
ds = load_from_disk(ds_path)
print(ds)

for key in ["instruction", "output", "text"]:
    inputs = tokenizer(ds["train"][key], padding="longest")
    print(key, np.array(inputs["input_ids"]).shape)

../dataset/DEBUG_N512_L16384


Saving the dataset (0/1 shards):   0%|          | 0/512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/512 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'text'],
        num_rows: 512
    })
    test: Dataset({
        features: ['instruction', 'output', 'text'],
        num_rows: 512
    })
})
instruction (512, 8193)
output (512, 8193)
text (512, 16385)
